In [ ]:
%pip install langchain openai cohere tiktoken

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/65/f4/dea10c11f0c2ebca298ba57f546940f8944e49c624239183c924ae0e1f81/langchain-0.0.339-py3-none-any.whl.metadata
  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/30/1d/27c3571504fb6fb1e9f7c906d93590ead22f5f34910489e155ee28512eeb/openai-1.3.5-py3-none-any.whl.metadata
  Obtaining dependency information for cohere from https://files.pythonhosted.org/packages/6c/b9/67b691fc5a8fa208435f85b38671d36238ab81eb1e7190c7c1e3b34892c7/cohere-4.36-py3-none-any.whl.metadata
  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/94/2f/0cc8fb3436d421d8fa2da370aca0283201f1b99e88a0f6e742bd8eef397d/tiktoken-0.5.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for dataclasses-json<0.7,>=0.5.7 from https://files.pythonhosted.org/packages/8d/e2/528c52001a743a7faa28e6d3095d9f01b472d3ef

In [1]:
import json
import random

example_file = open("../json/example.json","r")
test_file = open("../json/test.json","r")
format = open("../json/format.json","r").read()
tools = open("../json/tools.json","r").read()


In [2]:
# get examples
test_examples = json.loads(test_file.read())
examples = json.loads(example_file.read())

In [3]:
prompt_template= """
{tools_template}

{format_template}

{example_template}

{start_template}
"""

tools_template = """
Use the following tools and answer the given question using given answer format, 
{tools}
"""
answer_format_template = """
Generated answer for any given question should have following format
{format}
"""

example_template = """
Following are the example of question-answer pairs for the abovementioned tools
Example 1:
{example1}

Example 2:
{example2}

Whenver there is a need of finding current User api call to whoami is mandatory like following example

Example 3:
{example3}
"""

start_template = """
To reference the value of the ith tool in the chain, use $$PREV[i] as argument value. i = 0, 1, .. j-1; j = current tool’s index in the array
If the query could not be answered with the given set of tools, output an empty list instead.

Empty list should look like []. Whenever you want to access information about current user to pass it further, you need to make a call to whoami api tool.

Answer the following question based on instructions provided above.
question: {question}
answer:{{
"question":
"answer":
}}"""

In [4]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

full_prompt = PromptTemplate.from_template(prompt_template)
tools_prompt = PromptTemplate.from_template(tools_template)
answer_format_prompt = PromptTemplate.from_template(answer_format_template)
example_prompt = PromptTemplate.from_template(example_template)
start_prompt = PromptTemplate.from_template(start_template)

input_prompt = [
    ("tools_template",tools_prompt),
    ("format_template",answer_format_prompt),
    ("example_template",example_prompt),
    ("start_template",start_prompt)
]
prompt = PipelinePromptTemplate(final_prompt=full_prompt,pipeline_prompts=input_prompt)
prompt.input_variables

['tools', 'format', 'example2', 'example3', 'question', 'example1']

In [ ]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
config = open("config.txt","r").read()
OPENAI_API_KEY=config
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

chain = LLMChain(llm = ChatOpenAI(model='gpt-4'), prompt=prompt)

In [16]:
response = []
for test in test_examples:

    print(test['question_id'])
    # input = prompt.format(question=test['question'],tools=tools,format=format,examples=examples)
    output = chain.invoke({'question':test['question'],'tools':tools,'format':format,'example1':examples[0],'example2':examples[1],'example3':examples[2]})

    answer = json.loads(output['text'])
    if (answer == []):
        answer = {'question':test['question'],'answer':[]}
    answer['question_id'] = test['question_id']
    response.append(answer)


2
3
6
7
8


In [17]:
outfile = open("../output/zero shot with three examples/GPT_4_res.json","w")
json.dump(response,outfile,indent=4)
outfile.close()